### Student Information
Name: **Li Hin Yeung**

Student ID: **109164710**

GitHub ID: **BryanHY**

Kaggle name: **bryanlhy**

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2021-Lab2-master Repo](https://github.com/fhcalderon87/DM2021-Lab2-master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/c/dm2021-lab2-hw2/) regarding Emotion Recognition on Twitter. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Dec. 24th 11:59 pm, Friday)__. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 29th 11:59 pm, Wednesday)__. 

<div class="alert alert-block alert-info">
    <h1>Homework - Part 2 & 3</h1>
</div>

<div class="alert alert-block alert-warning">
<h2>Report of your work developing the model for the competition (You can use code and comment it).</h2>
    <h3>This report should include:</h3>
<ol>
    <li>what your preprocessing steps are</li>
    <li>the feature engineering steps</li>
    <li>an explanation of your model</li>
    <li>You can also mention different things you tried and insights you gained</li>
</ol>
</div>

# Import - Library

In [1]:
# TEST necessary for when working with external scripts
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import helpers.data_mining_helpers as dmh

import json

from tqdm.notebook import tqdm
tqdm.pandas()

# Import - Dataset

In [ ]:
json_tweets = []
for line in open('data/tweets_DM.json', 'r'):
    json_tweets.append(json.loads(line))

In [ ]:
df_dataId = pd.read_csv('data/data_identification.csv')

In [ ]:
df_emotion = pd.read_csv('data/emotion.csv')

# Create Dataframe - Intergate raw data

In [ ]:
# df_tweets = (
#     df_tweets["tweet"]
#     .apply(pd.Series)
#     .merge(df_tweets, left_index=True, right_index = True)
# )

In [5]:
### Create a dataframe of the tweets from tweets_DM.json
my_dict = {}
json_filtered_tweets = []

# json_decode = json.load(input_file)

### Only import the "tweet_id" and "text" from json
for item in json_tweets:
    my_dict={}
    my_dict['tweet_id']=item.get('_source').get('tweet').get('tweet_id')
    #  my_dict['hashtags']=item.get('_source').get('tweet').get('hashtags')
    my_dict['text']=item.get('_source').get('tweet').get('text')
    json_filtered_tweets.append(my_dict)


In [6]:
df_tweets = pd.DataFrame(json_filtered_tweets)

In [5]:
## save to pickle file
df_tweets.to_pickle("df_tweets_unprepro.pkl") 

In [2]:
import pandas as pd

## load a pickle file
df_tweets = pd.read_pickle("df_tweets_unprepro.pkl")

# Preprocessing

## - Divide dataset into training and testing

In [6]:
df_tweets

tweet_id                                               text
0        0x376b20  people post add snapchat must dehydrated . cuz...
1        0x2d5350  see , trump dangerous freepress around world ....
2        0x28b412  confident obedience , write , knowing even ask...
3        0x1cd5b0                            issa stalking tasha 😂😂😂
4        0x2de201  trust faith . friend someone trust . putting f...
...           ...                                                ...
1867530  0x316b80  buy last 2 tickets remaining show sell .. mixe...
1867531  0x29d0cb                swear hard work gone pay one day😈💰💸
1867532  0x2a6a4f                      card left n't idea get parcel
1867533  0x24faed  ah , corporate life , date using relative anac...
1867534  0x34be8c                         blessed living sundayvibes

[1867535 rows x 2 columns]

In [45]:
outer_join_df = pd.merge(df_tweets, df_dataId, on='tweet_id', how='outer')

In [46]:
train_df = outer_join_df.loc[outer_join_df.identification == 'train'].reset_index()

In [47]:
test_df = outer_join_df.loc[outer_join_df.identification == 'test'].reset_index()

## - Append training data's "emotion"

In [48]:
train_df = pd.merge(train_df, df_emotion, on='tweet_id', how='outer')

In [49]:
train_df = train_df.drop('index', axis=1)

In [50]:
test_df = test_df.drop('index', axis=1)

In [51]:
## save to pickle file
train_df.to_pickle("train_df_unprepo.pkl") 

test_df.to_pickle("test_df_unprepo.pkl") 

## - Cleaning data

In [2]:
import pandas as pd

## load a pickle file
train_df = pd.read_pickle("train_df_unprepo.pkl")
test_df = pd.read_pickle("test_df_unprepo.pkl")

In [ ]:
### Self-written data cleaning function

# import re

# def cleaning_text(text):
# #     text = re.sub(r'@\w+|#\w+|<>', '', text)
#     text = re.sub(r'http\S+', "", text)    # remove urls
#     text = re.sub(r'@\w+','',text)         # remove mentions
#     text = re.sub(r'#', '', text)       # remove hastags
#     text = re.sub(r'<.*?>','', text)       # remove html tags
    
#     # Should remove commas, fullstops too ??
    
#     return text

# df_tweets['text'] = df_tweets['text'].apply(lambda x: cleaning_text(x))

In [3]:
### External data cleaning functions found on the Internet

from bs4 import BeautifulSoup
import re
import itertools


def load_dict_smileys():
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }


def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "i'd":"i would",
        "i'll":"i will",
        "i'm":"i am",
        "i'm'a":"i am about to",
        "i'm'o":"i am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "i've":"i have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "whatcha":"what are you",
        "luv":"love",
        "sux":"sucks"
        }


def strip_accents(text):
    if 'ø' in text or  'Ø' in text:
        ## Do nothing when finding ø 
        return text   
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

In [4]:
def cleaning_part1(tweet):
    tweet = tweet.lower()
    CONTRACTIONS = load_dict_contractions()
    tweet = tweet.replace("’","'")
    words = tweet.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    tweet = " ".join(reformed)
    
    return tweet



def cleaning_part2(tweet):    
    
    ## Escaping HTML characters
    tweet = BeautifulSoup(tweet).get_text()
    
    ## Special case not handled previously.
    tweet = tweet.replace('\x92',"'")

    ## Standardizing words
    tweet = ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))
    
    ## Deal with smileys
    SMILEY = load_dict_smileys()  
    words = tweet.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    tweet = " ".join(reformed)
    
    tweet = re.sub(r'http\S+', ' ', tweet) # remove urls
    tweet = re.sub(r'@\w+',' ',tweet)      # remove mentions
    tweet = re.sub(r'#', ' ', tweet)       # remove hastags
    tweet = re.sub(r'<.*?>',' ', tweet)    # remove html tags
    tweet = re.sub(r'[\.\,\!\?\:\;\-\=\_\~]',' ', tweet)  # remove html tags

    ## Strip accents
    tweet = strip_accents(tweet)
    tweet = ' '.join(tweet.split())
    tweet = tweet.replace('"', ' ')

    return tweet

In [5]:
### Bertweet built-in normalizer
from helpers.TweetNormalizer import normalizeTweet


def bertweet_normalize(text):
    return normalizeTweet(text)

In [6]:
### Demonstration of the preprocessing procedures and the result after each function

n = 13

print(train_df.text[n])

part1 = cleaning_part1(train_df.text[n])
print(part1)

line = bertweet_normalize(part1)
print(line)

part2 = cleaning_part2(line)
print(part2)

line2 = bertweet_normalize(part2)
print(line2)

Progress at House Meyer, Pre Galv sub frame going in.  #maintenancefreesubframe #evatech <LH> #bendingcomposite #inovarsandtoncrew
progress at house meyer, pre galv sub frame going in. #maintenancefreesubframe #evatech <lh> #bendingcomposite #inovarsandtoncrew
progress at house meyer , pre galv sub frame going in . #maintenancefreesubframe #evatech <lh> #bendingcomposite #inovarsandtoncrew
progress at house meyer pre galv sub frame going in maintenancefreesubframe evatech bendingcomposite inovarsandtoncrew
progress at house meyer pre galv sub frame going in maintenancefreesubframe evatech bendingcomposite inovarsandtoncrew


In [7]:
### Cleaning training data
train_df['text'] = train_df['text'].progress_apply(lambda x: cleaning_part1(x))
train_df['text'] = train_df['text'].progress_apply(lambda x: bertweet_normalize(x))
train_df['text'] = train_df['text'].progress_apply(lambda x: cleaning_part2(x))
train_df['text'] = train_df['text'].progress_apply(lambda x: bertweet_normalize(x))

  0%|          | 0/1455563 [00:00<?, ?it/s]

  0%|          | 0/1455563 [00:00<?, ?it/s]

  0%|          | 0/1455563 [00:00<?, ?it/s]

  0%|          | 0/1455563 [00:00<?, ?it/s]

In [ ]:
test_df['text'] = test_df['text'].progress_apply(lambda x: cleaning_part1(x))
test_df['text'] = test_df['text'].progress_apply(lambda x: bertweet_normalize(x))
test_df['text'] = test_df['text'].progress_apply(lambda x: cleaning_part2(x))
test_df['text'] = test_df['text'].progress_apply(lambda x: bertweet_normalize(x))

<div class="alert alert-block alert-info">
    <h1>Feature Engineering</h1>
</div>

<div class="alert alert-block alert-warning">
    <h1>(a) BOW</h1>
</div>

In [59]:
from sklearn.feature_extraction.text import CountVectorizer

In [60]:
import nltk

# build analyzers (bag-of-words)
BOW = CountVectorizer(tokenizer=nltk.word_tokenize) 

In [61]:
# apply analyzer to training data
BOW.fit(train_df['text'])

/Users/bryan.li/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(tokenizer=<function word_tokenize at 0x7fd5e2102dc0>)

In [63]:
X_train = BOW.transform(train_df['text'])
y_train = train_df['emotion']

X_test = BOW.transform(test_df['text'])

In [64]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(1455563, 453451)
(1455563,)
(411972, 453451)


<div class="alert alert-block alert-warning">
    <h1>Classification - Naive Bayes</h1>
</div>

In [65]:
from sklearn.naive_bayes import MultinomialNB

mnb_tfidf = MultinomialNB()
mnb_tfidf.fit(X_train, y_train)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

MultinomialNB()

In [66]:
### Test result
test_result = mnb_tfidf.predict(X_test)

In [67]:
test_result.shape

(411972,)

# - Result (BOW + Naive Bayes)

- Scores vary with slightly different preprocessing procedures

<img src="img/bow_part1.png"/>

<img src="img/bow_part2.png"/>

---

<div class="alert alert-block alert-warning">
    <h1>(b) BOW 500</h1>
</div>

In [88]:
from sklearn.feature_extraction.text import CountVectorizer

In [89]:
import nltk

# build analyzers (bag-of-words)
BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize) 

In [90]:
# apply analyzer to training data
BOW_500.fit(train_df['text'])

/Users/bryan.li/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


CountVectorizer(max_features=500,
                tokenizer=<function word_tokenize at 0x7fb669de8cb0>)

In [91]:
"😂" in BOW_500.get_feature_names()

/Users/bryan.li/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


True

In [92]:
X_train = BOW_500.transform(train_df['text'])
y_train = train_df['emotion']

X_test = BOW_500.transform(test_df['text'])

In [93]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(1455563, 500)
(1455563,)
(411972, 500)


<div class="alert alert-block alert-warning">
    <h1>Classification - Naive Bayes</h1>
</div>

In [65]:
from sklearn.naive_bayes import MultinomialNB

mnb_tfidf = MultinomialNB()
mnb_tfidf.fit(X_train, y_train)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

MultinomialNB()

In [66]:
### Test result
test_result = mnb_tfidf.predict(X_test)

In [67]:
test_result.shape

(411972,)

# - Result (BOW_500 + Naive Bayes)
- Did not use Bertweet normalizer

<img src="img/bow_500.png" />

---

<div class="alert alert-block alert-warning">
    <h1>(c) BOW 1000</h1>
</div>

In [101]:
# import nltk

# build analyzers (bag-of-words)
BOW_1000 = CountVectorizer(max_features=1000, tokenizer=nltk.word_tokenize) 

In [102]:
# apply analyzer to training data
BOW_1000.fit(train_df['text'])

X_train = BOW_1000.transform(train_df['text'])
# y_train = train_df['emotion']

X_test = BOW_1000.transform(test_df['text'])

/Users/bryan.li/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


In [77]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(1455563, 250)
(1455563,)
(411972, 250)


<div class="alert alert-block alert-warning">
    <h1>Classification - Naive Bayes</h1>
</div>

In [65]:
from sklearn.naive_bayes import MultinomialNB

mnb_tfidf = MultinomialNB()
mnb_tfidf.fit(X_train, y_train)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

MultinomialNB()

In [66]:
### Test result
test_result = mnb_tfidf.predict(X_test)

In [67]:
test_result.shape

(411972,)

# - Result (BOW_1000 + Naive Bayes)

- BOW with feature limit does not generate a better result, due to certain features not being captured, thus decreases the performance of classification model.

<img src="img/bow_1000.png" />

---

<div class="alert alert-block alert-warning">
    <h1>(d) TFIDF Feature</h1>
</div>

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

tfidf_vector = TfidfVectorizer(max_features=500) 

In [60]:
tfidf_vector.fit(train_df['text'])

TfidfVectorizer(max_features=170000)

In [61]:
"😂" in tfidf_vector.get_feature_names()

False

In [62]:
X_train = tfidf_vector.fit_transform(train_df['text'])
#tfidf_vector.get_feature_names()

print(X_train.shape)

(1455563, 170000)


In [63]:
X_test = tfidf_vector.fit_transform(test_df['text'])
#tfidf_vector.get_feature_names()

print(X_test.shape)

(411972, 170000)


In [64]:
y_train = train_df['emotion']

<div class="alert alert-block alert-warning">
    <h1>(d1) Classification - Naive Bayes</h1>
</div>

In [65]:
from sklearn.naive_bayes import MultinomialNB

mnb_tfidf = MultinomialNB()
mnb_tfidf.fit(X_train, y_train)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

MultinomialNB()

In [66]:
### Test result
test_result = mnb_tfidf.predict(X_test)

In [67]:
test_result.shape

(411972,)

# - Result (TFIDF + Naive Bayes)

- Limitation: Without setting the maximum features, the dimensons/shapes of training data and testing data are different. Thus, it would not be able to utilize Naive Bayes for classification.

- Can see that using TF-IDF with max_features, the scores are lower than BOW with max_features. This could imply that depends some of the words appear in many documents, but they also still reflect certain sentiments of the writer.

<img src="img/tfidf.png" />

<div class="alert alert-block alert-warning">
    <h1>(d2) Logistic Regression</h1>
</div>

In [142]:
# from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split

In [143]:
classifier = LogisticRegression(solver='lbfgs',random_state=0)

In [144]:
classifier.fit(X_train, y_train)

/Users/bryan.li/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [145]:
test_result = classifier.predict(X_test)

# - Result (TFIDF + Logistic Regression)

- Using Logistic Regression, the performance is significantly worse than using Naive Bayes. This classification model might not be suited for sentinment analysis.

<img src="img/bow_log.png" />

---

<div class="alert alert-block alert-info">
    <h1>Export result as CSV</h1>
</div>

In [ ]:
## Execute this if there are previously existing results
#test_df = test_df.drop('emotion', axis=1)

In [69]:
test_df.insert(3, 'emotion', test_result)

In [70]:
test_df

tweet_id                                               text  \
0       0x28b412  confident of your obedience i write to you kno...   
1       0x2de201  trust is not the same as faith a friend is som...   
2       0x218443  when do you have enough when are you satisfied...   
3       0x2939d5  god woke you up now chase the day godsplan god...   
4       0x26289a  in these tough times who do you turn to as you...   
...          ...                                                ...   
411967  0x2913b4  for this is the message that ye heard from the...   
411968  0x2a980e  there is a lad here which hath five barley loa...   
411969  0x316b80  when you buy the last 2 tickets remaining for ...   
411970  0x29d0cb  i swear all this hard work gone pay off one da...   
411971  0x2a6a4f  no card left when i was not in so i have no id...   

       identification       emotion  
0                test  anticipation  
1                test  anticipation  
2                test  anticipation  
3                test  anticipation  
4                test  anticipation  
...               ...           ...  
411967           test  anticipation  
411968           test  anticipation  
411969           test           joy  
411970           test           joy  
411971           test       sadness  

[411972 rows x 4 columns]

In [71]:
## Remove the "text" and 
result_df = test_df.drop(['text', 'identification'], axis=1)

In [72]:
result_df.rename(columns={'tweet_id': 'id'}, inplace=True)

In [74]:
result_df.to_csv('submission.csv', index=False)

---

<div class="alert alert-block alert-danger">
    <h1>Not fully tried models (Due to hardware limitations)</h1>
</div>

Below there are 2 models that I was not able to utilize for this homework, due to hardware limitations. The feature engineering / training procedures would take a very long duration to complete and before completion my notebook begun to overheat.

<div class="alert alert-block alert-warning">
    <h1>Bertweet transformer</h1>
</div>

In [ ]:
import torch
from transformers import AutoTokenizer
from helpers.TweetNormalizer import normalizeTweet

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large")

line = normalizeTweet("DHEC confirms https://postandcourier.com/health/covid19/sc-has-first-two-presumptive-cases-of-coronavirus-dhec-confirms/article_bddfe4ae-5fd3-11ea-9ce4-5f495366cee6.html?utm_medium=social&utm_source=twitter&utm_campaign=user-share… via @postandcourier 😢")

input_ids = torch.tensor([tokenizer.encode(line)])

with torch.no_grad():
    features = bertweet(input_ids)  # Models outputs are now tuples

In [ ]:
print(features[0].shape)
print(features[1].shape)

In [13]:
import torch
from transformers import AutoModel, AutoTokenizer
from helpers.TweetNormalizer import normalizeTweet

bertweet = AutoModel.from_pretrained("vinai/bertweet-large")

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large")


def bertweet_transformer(line):
    input_ids = torch.tensor([tokenizer.encode(line)])

    with torch.no_grad():
        features = bertweet(input_ids)  # Models outputs are now tuples
        return features[1]

Some weights of the model checkpoint at vinai/bertweet-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use 

In [14]:
### Demonstration on how the feature vector looks like
line = train_df.text[10]
bertweet_transformer(line)

tensor([[ 0.5160,  0.4107,  0.2241,  ..., -0.2170,  0.1064,  0.3342]])

In [ ]:
### This step was not completed (due to hardware limitations)
train_df['bertweet_features'] = train_df['betweet_normalize'].progress_apply(lambda x: bertweet_transformer(x))
test_df['bertweet_features'] = test_df['betweet_normalize'].progress_apply(lambda x: bertweet_transformer(x))

-------

<div class="alert alert-block alert-warning">
    <h1>Deep Learning</h1>
</div>

In [28]:
## deal with label (string -> one-hot)
import keras

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)

Using TensorFlow backend.


check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 0    anticipation
1         sadness
2            fear
3             joy
Name: emotion, dtype: object

y_train.shape:  (1455563,)


## After convert
y_train[0:4]:
 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]

y_train.shape:  (1455563, 8)


In [29]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  500
output_shape:  8


In [30]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                32064     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
re_lu_2 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 520       
_________________________________________________________________
softmax_1 (Softmax)          (None, 8)                 0   

2021-12-22 00:21:13.828491: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-22 00:21:13.829810: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [34]:
### Did not complete the training (Due to hardware limitation)

from keras.callbacks import CSVLogger

# csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 25
batch_size = 32

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size)
print('training finish')


Epoch 1/25
1455563/1455563 [==============================] - 231s 159us/step - loss: 1.4958 - accuracy: 0.4536
Epoch 2/25
1455563/1455563 [==============================] - 221s 152us/step - loss: 1.4674 - accuracy: 0.4631
Epoch 3/25
1455563/1455563 [==============================] - 206s 141us/step - loss: 1.4598 - accuracy: 0.4654
Epoch 4/25
1455563/1455563 [==============================] - 211s 145us/step - loss: 1.4553 - accuracy: 0.4670
Epoch 5/25
1455563/1455563 [==============================] - 217s 149us/step - loss: 1.4524 - accuracy: 0.4683
Epoch 6/25
 397536/1455563 [=======>......................] - ETA: 2:31 - loss: 1.4474 - accuracy: 0.4699

KeyboardInterrupt: 